In [ ]:
#/content/drive/MyDrive/Colab Notebooks/Deep learning/ANN/Day 1/Churn_Modelling.csv

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df=pd.read_csv("Churn_Modelling.csv")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
X=df[['CreditScore','Geography','Gender','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary']]

In [ ]:
y=df['Exited']

In [ ]:
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [ ]:
geography=pd.get_dummies(X['Geography'],drop_first=True)

In [ ]:
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X["Gender"],drop_first=True)

X=pd.concat([X,geography,gender],axis=1)


In [ ]:
X.head()

In [ ]:
X=X.drop(X[['Geography','Gender']],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

**Feature Scaling**

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
classifier=Sequential()

In [ ]:
classifier.add(Dense(units=6,kernel_initializer='he_uniform',activation='relu',input_dim=11))

In [ ]:
classifier.add(Dense(units=6,activation='relu',kernel_initializer='he_uniform'))

In [ ]:
classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [ ]:
classifier.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
model_history=classifier.fit(X_train, y_train,validation_split=0.33, batch_size = 10, epochs = 100)

In [ ]:
y_pred=classifier.predict(X_test)

In [ ]:
y_pred=(y_pred>0.5)

In [ ]:
from sklearn.metrics import  confusion_matrix,classification_report

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cr=classification_report(y_test,y_pred)

In [ ]:
print(cm)

In [ ]:
print(cr)

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)
print(score)

# HYPERPARAMETER OPTIMIZATION

In [ ]:
pip install -U keras-tuner

In [ ]:
from tensorflow.keras import  layers
from kerastuner.tuners import RandomSearch

In [ ]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(units=hp.Int('units',min_value=32,
                               max_value=512,
                               step=32),
                  activation='relu'))
  model.add(Dense(1,activation='relu'))
  model.compile(optimizer=keras.optimizers.Adam(
      hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])),
      loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
  return model

In [ ]:
tuner=RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='output',
    project_name='day_1'
)

In [ ]:
tuner.search_space_summary()

In [ ]:
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_split=0.1)

In [ ]:
model=tuner.get_best_models(num_models=2)

In [ ]:
tuner.results_summary()

In [ ]:
classifier_opti=Sequential()

In [ ]:
# Adding the input layer and the first hidden layer
classifier_opti.add(Dense(units = 256, kernel_initializer = 'he_uniform',activation='relu',input_dim = 11))

# Adding the second hidden layer
classifier_opti.add(Dense(units = 6, kernel_initializer = 'he_uniform',activation='relu'))
# Adding the output layer
classifier_opti.add(Dense(units = 1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier_opti.compile(optimizer = 'Adamax', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
model_history=classifier_opti.fit(X_train, y_train,validation_split=0.33, batch_size = 10, epochs = 1000)

In [ ]:
y_pred_new=classifier.predict(X_test)

In [ ]:
y_pred_new=(y_pred_new>0.5)

In [ ]:
from sklearn.metrics import  confusion_matrix,classification_report

In [ ]:
cm=confusion_matrix(y_test,y_pred_new)
cr=classification_report(y_test,y_pred_new)

In [ ]:
print(cm)

In [ ]:
print(cr)

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_pred_new)
print(score)

# **Deployment**